In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import gc
import glob

# 그래프 기본 테마 설정
# https://coldbrown.co.kr/2023/07/%ED%8C%8C%EC%9D%B4%EC%8D%AC-%EC%8B%A4%EC%A0%84%ED%8E%B8-08-seaborn-sns-set%EC%9D%84-%ED%86%B5%ED%95%B4-%EC%8A%A4%ED%83%80%EC%9D%BC-%EC%84%A4%EC%A0%95%ED%95%98%EA%B8%B0/
sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False


# 복잡한 통계 처리를 위한 라이브러리
from scipy import stats

In [2]:
# data_type = "train"
# month = "07"
# category = "잔액정보"

# local
root_path = '../../data'

# colab
# root_path = '/content/drive/MyDrive/12조 파이널프로젝트/data'

#drive_folder = f'{root_path}/스탯티즈_Raw_data/'

In [3]:
df1 = pd.read_csv(f'{root_path}/KBO FA 투수 [2013-2024]_최종_전처리.csv', encoding='utf-8')

## '피장타율' 파생 컬럼
- 선수명, 플레이년도만 엑셀파일로 저장
- 수기로 피장타율 입력
- 다시 가져와서 선수명, 플레이년도로 merge

In [5]:
# template_df = df1[['선수명', '플레이년도']].drop_duplicates().sort_values(['선수명', '플레이년도'])
# template_df['피장타율_입력'] = None  # 수기로 채울 컬럼 추가

# template_df.to_excel("../../data/피장타율_입력템플릿.xlsx", index=False)

## '완투' / '완봉' 파생 컬럼

In [7]:
# template_df2 = df1[['선수명', '플레이년도']].drop_duplicates().sort_values(['선수명', '플레이년도'])
# template_df2['완투_입력'] = None  # 수기로 채울 컬럼 추가
# template_df2['완봉_입력'] = None  # 수기로 채울 컬럼 추가

# template_df2.to_excel("../../data/완투완봉_입력템플릿.xlsx", index=False)

## '사사구' 파생 컬럼
- 볼넷 + 사구

In [9]:
df1['사사구'] = df1['볼넷허용'] + df1['사구허용']

In [10]:
df1['사사구']

0      25.0
1      24.0
2      19.0
3      21.0
4      26.0
       ... 
360    59.0
361    47.0
362    18.0
363    33.0
364    16.0
Name: 사사구, Length: 365, dtype: float64

## '원클럽맨' 파생 컬럼
- 같은 선수명이 한 구단에서만 활동한 경우

In [12]:
# # 선수명 기준으로 몇 개의 구단에서 뛰었는지 count
# club_counts = df1.groupby('선수명')['구단명'].nunique()

# # 원클럽맨 여부 판단: 구단 수가 1인 경우 True
# df1['원클럽맨'] = df1['선수명'].map(lambda name: club_counts[name] == 1)

In [13]:
# df1['원클럽맨'].value_counts()

## '투구수' 파생 컬럼
- 나중에 필요시, 이닝으로 나누면 '게임당 투구수'

In [15]:
# template_df3 = df1[['선수명', '플레이년도']].drop_duplicates().sort_values(['선수명', '플레이년도'])
# template_df3['투구수_입력'] = None  # 수기로 채울 컬럼 추가

# template_df3.to_excel("../../data/투구수_입력템플릿.xlsx", index=False)

## Merge

In [17]:
# ✔ 완투완봉
df_wan = pd.read_excel('../../data/완투완봉_입력템플릿.xlsx')
df_wan = df_wan.rename(columns={
    '완투_입력': '완투',
    '완봉_입력': '완봉'
})

# ✔ 투구수
df_pitch = pd.read_excel('../../data/투구수_입력템플릿.xlsx')
df_pitch = df_pitch.rename(columns={
    '투구수_입력': '투구수'
})

# ✔ 피장타율
df_slg = pd.read_excel('../../data/피장타율_입력템플릿.xlsx')
df_slg = df_slg.rename(columns={
    '피장타율_입력': '피장타율'
})

# ✔ 원클럽맨여부
df_club = pd.read_csv('../../data/FA투수데이터_원클럽맨여부.csv')

# 필요한 컬럼만 선택
df_club = df_club[['선수명', '플레이년도', '원클럽맨여부']]

In [18]:
df1 = df1.merge(df_wan, on=['선수명', '플레이년도'], how='left')
df1 = df1.merge(df_pitch, on=['선수명', '플레이년도'], how='left')
df1 = df1.merge(df_slg, on=['선수명', '플레이년도'], how='left')
df1 = df1.merge(df_club, on=['선수명', '플레이년도'], how='left')

## 비FA 선수들의 원클럽맨여부

In [43]:
df1.loc[df1['선수명'] == '문승원', '원클럽맨여부'] = 1
df1.loc[df1['선수명'] == '박종훈', '원클럽맨여부'] = 1
df1.loc[df1['선수명'] == '박세웅', '원클럽맨여부'] = 0
df1.loc[df1['선수명'] == '김광현', '원클럽맨여부'] = 1
df1.loc[df1['선수명'] == '류현진', '원클럽맨여부'] = 1
df1.loc[df1['선수명'] == '김상수', '원클럽맨여부'] = 0
df1.loc[df1['선수명'] == '고영표', '원클럽맨여부'] = 1
df1.loc[df1['선수명'] == '구창모', '원클럽맨여부'] = 1

In [45]:
print(df1.columns)
print(df1.isna().sum())  # 병합된 컬럼들에 결측치 있는지 확인

Index(['FA년도', '구단명', '선수명', '세부 포지션', '투', '구분', 'FA등급', '플레이년도', '나이', '게임수',
       '선발등판', '구원등판', '승리', '패전', '세이브', '블론세이브', '홀드', '블론홀드', '이닝수', '자책점',
       '피안타', '2루타허용', '3루타허용', '피홈런', '볼넷허용', '사구허용', '탈삼진', '폭투', 'ERA',
       'FIP', '선발 이닝', '구원 이닝', '선발 WAR', '구원 WAR', '종합 WAR', '연봉', '비고1',
       '잔류 여부', 'FA 계약 연수', 'FA 계약 총액', '사사구', '완투', '완봉', '투구수', '피장타율',
       '원클럽맨여부'],
      dtype='object')
FA년도        0
구단명         0
선수명         0
세부 포지션      0
투           0
구분          0
FA등급        0
플레이년도       0
나이          0
게임수         0
선발등판        0
구원등판        0
승리          0
패전          0
세이브         0
블론세이브       0
홀드          0
블론홀드        0
이닝수         0
자책점         0
피안타         0
2루타허용       0
3루타허용       0
피홈런         0
볼넷허용        0
사구허용        0
탈삼진         0
폭투          0
ERA         0
FIP         0
선발 이닝       0
구원 이닝       0
선발 WAR      0
구원 WAR      0
종합 WAR      0
연봉          0
비고1         0
잔류 여부       0
FA 계약 연수    0
FA 계약 총액    0
사사구         0
완투 

In [47]:
df1.to_csv('../../data/KBO FA 투수 [2013-2024]_최종_전처리+파생.csv', index=False, encoding='utf-8-sig')